In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [13]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch

# 모델 및 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터셋 로드
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["text"], train_data["target"], test_size=0.2, random_state=42
)

# 데이터 전처리 함수 정의
def preprocess_data(texts, labels=None):
    encodings = tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=128,
    )
    if labels is not None:
        encodings["labels"] = list(labels)
    return encodings

# 데이터 토큰화
train_encodings = preprocess_data(train_texts, train_labels)
val_encodings = preprocess_data(val_texts, val_labels)

# Hugging Face 데이터셋 포맷으로 변환
train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)

# 평가 메트릭 함수 정의
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 모델 학습
trainer.train()

# 검증 데이터 예측
results = trainer.evaluate()
print("Validation Accuracy:", results["eval_accuracy"])

# 테스트 데이터 토큰화
test_encodings = preprocess_data(test_data["text"])
test_dataset = Dataset.from_dict(test_encodings)

# 테스트 데이터 예측
predictions = trainer.predict(test_dataset)
test_data["target"] = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

# 제출 파일 생성
submission = test_data[["id", "target"]]
submission.to_csv("submission_v2_BERT_v2.csv", index=False)
print("Submission file saved as 'submission.csv'")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-2064381da7c2>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.365900,0.408391,0.823375
2,0.258500,0.454975,0.837164
3,0.229600,0.579744,0.831254


Validation Accuracy: 0.837163507938385
Submission file saved as 'submission.csv'


In [2]:
# 그리드 서치

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch

# 모델 및 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# 데이터셋 로드
train_data = pd.read_csv("train.csv")

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["text"], train_data["target"], test_size=0.2, random_state=42
)

# 데이터 전처리 함수 정의
def preprocess_data(texts, labels=None):
    encodings = tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=128,
    )
    if labels is not None:
        encodings["labels"] = list(labels)
    return encodings

# 데이터 토큰화
train_encodings = preprocess_data(train_texts, train_labels)
val_encodings = preprocess_data(val_texts, val_labels)

# Hugging Face 데이터셋 포맷으로 변환
train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

# 평가 메트릭 함수 정의
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
# 하이퍼파라미터 그리드 정의
grid = {
    "learning_rate": [1e-5, 2e-5, 3e-5, 4e-5, 5e-5],  # 학습률
    "per_device_train_batch_size": [8, 16],  # 배치 크기
    "num_train_epochs": [2, 3, 4, 5],  # 에포크 수
}

# 최적의 하이퍼파라미터를 저장할 변수
best_accuracy = 0
best_params = {}

# 그리드 서치 실행
for learning_rate in grid["learning_rate"]:
    for batch_size in grid["per_device_train_batch_size"]:
        for num_epochs in grid["num_train_epochs"]:
            print(f"Testing config: lr={learning_rate}, batch_size={batch_size}, epochs={num_epochs}")

            # 학습 설정
            training_args = TrainingArguments(
                output_dir="./results",
                evaluation_strategy="epoch",
                save_strategy="no",  # 모델 저장 생략
                num_train_epochs=num_epochs,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                learning_rate=learning_rate,
                logging_dir="./logs",
                logging_steps=10,
                report_to="none"
            )

            # 모델 초기화
            model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

            # Trainer 설정
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
            )

            # 모델 학습
            trainer.train()

            # 검증 데이터 평가
            results = trainer.evaluate()
            accuracy = results["eval_accuracy"]
            print(f"Accuracy: {accuracy}")

            # 최적의 하이퍼파라미터 업데이트
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    "learning_rate": learning_rate,
                    "batch_size": batch_size,
                    "num_train_epochs": num_epochs,
                }

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:")
print(best_params)
print(f"Best Validation Accuracy: {best_accuracy}")

Testing config: lr=1e-05, batch_size=8, epochs=2


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-dce75ed50f05>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.319300,0.397482,0.833880
2,0.241000,0.466247,0.835194


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.332800,0.387581,0.837820
2,0.285700,0.427014,0.845043
3,0.225500,0.588362,0.810243
4,0.115600,0.713618,0.826658


Accuracy: 0.8266578912734985
Testing config: lr=5e-05, batch_size=16, epochs=5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-dce75ed50f05>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.310400,0.405158,0.831254
2,0.273800,0.472634,0.826001
3,0.264900,0.702293,0.796454
4,0.113500,0.748990,0.812869
5,0.107800,0.870414,0.820749


Accuracy: 0.820748507976532
Best Hyperparameters:
{'learning_rate': 2e-05, 'batch_size': 16, 'num_train_epochs': 3}
Best Validation Accuracy: 0.847012460231781


In [4]:
# 최적의 하이퍼파라미터로 모델 재학습
print("Training with Best Hyperparameters...")
best_learning_rate = best_params["learning_rate"]
best_batch_size = best_params["batch_size"]
best_num_epochs = best_params["num_train_epochs"]

# 학습 설정
final_training_args = TrainingArguments(
    output_dir="./final_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=best_num_epochs,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    learning_rate=best_learning_rate,
    logging_dir="./final_logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)

# 모델 초기화
final_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Trainer 설정
final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 모델 재학습
final_trainer.train()

# 검증 데이터 평가
final_results = final_trainer.evaluate()
print("Final Validation Accuracy:", final_results["eval_accuracy"])

# 테스트 데이터 예측
print("Predicting on Test Data...")
test_encodings = preprocess_data(test_data["text"])
test_dataset = Dataset.from_dict(test_encodings)

test_predictions = final_trainer.predict(test_dataset)
test_data["target"] = torch.argmax(torch.tensor(test_predictions.predictions), axis=1).numpy()

# 제출 파일 생성
submission = test_data[["id", "target"]]
submission.to_csv("submission_v2_BERT_GridSearch.csv", index=False)
print("Submission file saved as 'submission.csv'")


Training with Best Hyperparameters...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-0c166add2440>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.309300,0.389584,0.839133
2,0.309000,0.424271,0.837164
3,0.319300,0.482708,0.837820


Final Validation Accuracy: 0.8391332626342773
Predicting on Test Data...


NameError: name 'test_data' is not defined

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch

# 모델 및 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터셋 로드
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["text"], train_data["target"], test_size=0.2, random_state=42
)

# 데이터 전처리 함수 정의
def preprocess_data(texts, labels=None):
    encodings = tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=128,
    )
    if labels is not None:
        encodings["labels"] = list(labels)
    return encodings

# 데이터 토큰화
train_encodings = preprocess_data(train_texts, train_labels)
val_encodings = preprocess_data(val_texts, val_labels)

# Hugging Face 데이터셋 포맷으로 변환
train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-05,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)

# 평가 메트릭 함수 정의
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 모델 학습
trainer.train()

# 검증 데이터 예측
results = trainer.evaluate()
print("Validation Accuracy:", results["eval_accuracy"])

# 테스트 데이터 토큰화
test_encodings = preprocess_data(test_data["text"])
test_dataset = Dataset.from_dict(test_encodings)

# 테스트 데이터 예측
predictions = trainer.predict(test_dataset)
test_data["target"] = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

# 제출 파일 생성
submission = test_data[["id", "target"]]
submission.to_csv("submission_v2_BERT_GridSearch.csv", index=False)
print("Submission file saved as 'submission.csv'")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-33ba5a2c3287>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.351900,0.387700,0.838477
2,0.299400,0.417934,0.843729
3,0.281600,0.475705,0.842416


Validation Accuracy: 0.8437294960021973
Submission file saved as 'submission.csv'
